<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Select-events" data-toc-modified-id="Select-events-0.1">Select events</a></span><ul class="toc-item"><li><span><a href="#Save-selected-events-to-file" data-toc-modified-id="Save-selected-events-to-file-0.1.1">Save selected events to file</a></span></li></ul></li><li><span><a href="#Align-clusters" data-toc-modified-id="Align-clusters-0.2">Align clusters</a></span></li><li><span><a href="#Add-tracks-and-vertex-to-events" data-toc-modified-id="Add-tracks-and-vertex-to-events-0.3">Add tracks and vertex to events</a></span><ul class="toc-item"><li><span><a href="#Save-tracked-events-to-file" data-toc-modified-id="Save-tracked-events-to-file-0.3.1">Save tracked events to file</a></span></li><li><span><a href="#Opening-angle" data-toc-modified-id="Opening-angle-0.3.2">Opening angle</a></span></li><li><span><a href="#DCA-between-two-tracks" data-toc-modified-id="DCA-between-two-tracks-0.3.3">DCA between two tracks</a></span></li><li><span><a href="#DCA-between-vertex-and-[0,0,0]" data-toc-modified-id="DCA-between-vertex-and-[0,0,0]-0.3.4">DCA between vertex and [0,0,0]</a></span></li></ul></li></ul></li><li><span><a href="#Very-clean-events" data-toc-modified-id="Very-clean-events-1">Very clean events</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Vertex-positions" data-toc-modified-id="Vertex-positions-1.0.1">Vertex positions</a></span></li><li><span><a href="#Position-shift-between-layers" data-toc-modified-id="Position-shift-between-layers-1.0.2">Position shift between layers</a></span></li><li><span><a href="#Correlation-between-ALPIDEs-after-all-selections" data-toc-modified-id="Correlation-between-ALPIDEs-after-all-selections-1.0.3">Correlation between ALPIDEs after all selections</a></span></li></ul></li></ul></li></ul></div>

In [2]:
import math
import matplotlib.pyplot as plt
import ROOT

import numpy as np
from uits3_krakow22.src.Event import Event
from uits3_krakow22.src.Cluster import Cluster
from uits3_krakow22.src.Track import Track
from uits3_krakow22.src.Vertex import Vertex
from uits3_krakow22.src.Utils import *

from ipywidgets import IntProgress
import matplotlib.pyplot as plt
from IPython.display import display,HTML,clear_output
display(HTML("<style>.container { width:95% !important; }</style>"))
display(HTML("<style>table {float:left;}</style>"))

Welcome to JupyROOT 6.24/06


### Select events (and prealign azimuth)

In [3]:
#first weekend files

#path1 = "/home/cehrich/Software/uits3_krakow22/noFlipNeeded/run447162516_221106162754.pkl" #200 MeV,14.9Gb, threshold not mentioned Run 57
#path2 = "/home/cehrich/Software/uits3_krakow22/noFlipNeeded/run445203522_221104212842.pkl" #120 MeV, 2.3Gb, threshold not mentioned Run 031, may be junk
#path3 = "/home/cehrich/Software/uits3_krakow22/noFlipNeeded/run446172943_221105174846.pkl" #80 MeV,  2.2Gb, threshold not mentioned Run 38


#second weekend
#path = "/home/cehrich/Software/uits3_krakow22/noFlipNeeded/run456103757_221112104316.pkl"  #120MeV 300e 
#path = "/home/cehrich/Software/uits3_krakow22/noFlipNeeded/run456114514_221112120005.pkl" #80MeV 100e 
path = "/home/cehrich/Software/uits3_krakow22/pickleNeedsFlip/run456195948_221112200957.pkl" #200MeV 100e


#run456114514_221112120005.pkl"
  #uits3_krakow22/output/output.pkl" #yzAlign_run456114514_221112120005.pkl" #"/home/cehrich/Software/uits3_krakow22/run457063715_221113063928_clusters.pkl" #"/home/cehrich/Software/uits3_krakow22/run456195948_221112200957.pkl" 
#first file is 80MeV, 0.6 GB, second and third are 200 MeV at 3GB and 12GB respectively (pkl size)
nEvents = 1e7#-1
selectedEvents = []
if nEvents > 0 :
    f = IntProgress(min=0, max=nEvents, description="Processing:")
    display(f)

shift = -1    
    
for i,event in enumerate(readEvents(path, nEvents=nEvents)):
    #print("test")
    if nEvents > 0 and i>0 and i%1e4 == 0: f.value += 1e4
    eventStatus = True
    #event.clusters = [cluster.flipYaxis() for cluster in event.clusters]
    event.clusters = [cluster.shiftXaxis("ALPIDE_1",shift, "angle") for cluster in event.clusters if 23 > cluster.size > 3]
        
    for alpide in ["ALPIDE_0","ALPIDE_1","ALPIDE_2","ALPIDE_3","ALPIDE_4"]:
        clusters = event.selectDetector(alpide)
        if len(clusters) != 1:
            eventStatus = False
            break
    if eventStatus : selectedEvents.append(event)
        
if 'f' in locals(): 
    f.bar_style = "success"
    print("Found events:",len(selectedEvents),", Fraction:",len(selectedEvents)*100/nEvents,"%")

IntProgress(value=0, description='Processing:', max=10000000)

Going to process 10000000.0 events.
Found events: 14603 , Fraction: 0.14603 %


#### Save selected events to file

In [4]:
path_selected = "/home/cehrich/Software/uits3_krakow22/selectedEvents.pkl"
save2pickle(selectedEvents,path_selected)

### Align clusters

In [5]:
displacements = {
        "ALPIDE_0" : [0,0,0],
        "ALPIDE_1" : [1.0, 0, 0],
        "ALPIDE_2" : [0,0,0],
        "ALPIDE_3" : [0.1875, 0.1875,0],
        "ALPIDE_4" : [0.0, -0.03125, 0] 
}

displacements0 = {
    "ALPIDE_0" : [0,0,0], # anchor
    "ALPIDE_1" : [1.156+0.05531,0.2837-0.03655,0],
    "ALPIDE_2" : [0.06475+0.2488,0.08900-0.03316,0],
    "ALPIDE_3" : [1.426-0.1012,0.7004-1.116,0],
    "ALPIDE_4" : [0.5261+0.5159,1.102-1.372,0],
}

displacements1 = {
    "ALPIDE_0" : [-1.211,-0.2472,0], 
    "ALPIDE_1" : [0,0,0],# anchor
    "ALPIDE_2" : [-0.7214,0.4154,0],
    "ALPIDE_3" : [0.4301,-0.02330,0],
    "ALPIDE_4" : [0.48375,0.01058,0],
}

displacements2 = {
    "ALPIDE_0" : [-0.3135,-0.05585,0], 
    "ALPIDE_1" : [0.7214,-0.4154,0],
    "ALPIDE_2" : [0,0,0],# anchor
    "ALPIDE_3" : [0.8794,-0.1445,0],
    "ALPIDE_4" : [0.9993,-0.1880,0],
}

displacements3 = {
    "ALPIDE_0" : [-1.321,0.4156,0], 
    "ALPIDE_1" : [-0.4301,0.02330,0],
    "ALPIDE_2" : [-0.8794,0.1445,0],
    "ALPIDE_3" : [0,0,0],# anchor
    "ALPIDE_4" : [-0.1492,0.8093,0],
}

displacements4 = {
    "ALPIDE_0" : [-1.042,0.2698,0], 
    "ALPIDE_1" : [-0.4837,-0.01058,0],
    "ALPIDE_2" : [-0.9960,0.1885,0],
    "ALPIDE_3" : [0.1492,-0.80930,0],
    "ALPIDE_4" : [0,0,0],# anchor
}

displacements5 = {
        "ALPIDE_0" : [0.0, -0.890625, 0],
        "ALPIDE_1" : [1.03125, -0.96875, 0],
        "ALPIDE_2" : [0,0,0],
        "ALPIDE_3" : [0.1875, 0.1875,0],
        "ALPIDE_4" : [0.0, -0.03125, 0],
}

displacements6 = {
        "ALPIDE_0" : [0,0,0],
        "ALPIDE_1" : [0,0,0],
        "ALPIDE_2" : [0,0,0],
        "ALPIDE_3" : [0,0,0],
        "ALPIDE_4" : [0,0,0]
}

displacements7 = {
        "ALPIDE_0" : [0.0, -1.0, 0],
        "ALPIDE_1" : [1.03125, -0.96875, 0],
        "ALPIDE_2" : [0, 0, 0],
        "ALPIDE_3" : [0.1875, 0.1875, 0],
        "ALPIDE_4" : [0.0, -0.03125, 0]
}

displacements8 = {
        "ALPIDE_0" : [0,0,0],
        "ALPIDE_1" : [0,0,0],
        "ALPIDE_2" : [0.44062500000000004, -0.5703125, 0],
        "ALPIDE_3" : [0,0,0],
        "ALPIDE_4" : [0,0,0]
}

displacements9 = {
        "ALPIDE_0" : [-0.5, 0.5, 0],
        "ALPIDE_1" : [0,0,0],
        "ALPIDE_2" : [0,0,0],
        "ALPIDE_3" : [0,0,0],
        "ALPIDE_4" : [0,0,0]
}

displacements10 = {
        "ALPIDE_0" : [0,0,0],
        "ALPIDE_1" : [0,0,0],
        "ALPIDE_2" : [0,0,0],
        "ALPIDE_3" : [0.5, 0.5, 0],
        "ALPIDE_4" : [0,0,0]
}

displacements11 = {
        "ALPIDE_0" : [0,0,0],
        "ALPIDE_1" : [0,0,0],
        "ALPIDE_2" : [0,0,0],
        "ALPIDE_3" : [0,0,0],
        "ALPIDE_4" : [-0.71875, -0.71875, 0]
}

displacements12 = {
        "ALPIDE_0" : [0,0,0],
        "ALPIDE_1" : [1,-1,0],
        "ALPIDE_2" : [0,0,0],
        "ALPIDE_3" : [0,0,0],
        "ALPIDE_4" : [0,0,0]
}

displacements_round6 = {
        "ALPIDE_0" : [-1, -1, 0], 
        "ALPIDE_1" :[1, 1, 0], 
        "ALPIDE_2" :[-1, 1, 0], 
        "ALPIDE_3" :[1, -1, 0], 
        "ALPIDE_4" :[1, 1, 0]
}

displacements_round5 = {
        "ALPIDE_0" : [0, -1, 0], 
        "ALPIDE_1" :[0, 0, 0], 
        "ALPIDE_2" :[0, 0, 0], 
        "ALPIDE_3" :[1, 0, 0], 
        "ALPIDE_4" :[1, 0, 0]
}

displacements_round4 = {
        "ALPIDE_0" : [0, -1, 0], 
        "ALPIDE_1" :[0, 0, 0], 
        "ALPIDE_2" :[0, 0, 0], 
        "ALPIDE_3" :[1, 0, 0], 
        "ALPIDE_4" :[1, 0, 0]
}

displacements_round3 = {
        "ALPIDE_0" :[0.0, 0.25, 0], 
        "ALPIDE_1" :[0.75, 0.25, 0], 
        "ALPIDE_2" :[0.0, 0.0, 0], 
        "ALPIDE_3" :[0.75, 0.25, 0], 
        "ALPIDE_4" :[0.75, 0.0, 0]
}

displacements_round2= {
        "ALPIDE_0" :[0.0, 0.25, 0], 
        "ALPIDE_1" :[0.75, 0.25, 0], 
        "ALPIDE_2" :[0.0, 0.0, 0], 
        "ALPIDE_3" :[0.75, 0.25, 0], 
        "ALPIDE_4" :[0.75, 0.0, 0]
}

displacements_round1 = {
        "ALPIDE_0" :[0.0, 0.0625, 0], 
        "ALPIDE_1" :[0.75, 0.25, 0],
        "ALPIDE_2" :[0.0, 0.0, 0],  
        "ALPIDE_3" :[0.75, 0.25, 0], 
        "ALPIDE_4" :[0.75, 0.0, 0]
}
#Got median DCA  0.124  and mean DCA:  0.171  and DCA sigma:  0.198

displacements_round0 = {
        "ALPIDE_0" :[0.0, 0.03125, 0],
        "ALPIDE_1" :[0.78125, 0.03125, 0],
        "ALPIDE_2" :[0.0, 0.03125, 0], 
        "ALPIDE_3" :[0.75, 0.03125, 0],
        "ALPIDE_4" :[0.78125, 0.03125, 0]
}
#Got median DCA  0.123  and mean DCA:  0.170  and DCA sigma:  0.197
   

displacements_round0200MeV = {
        "ALPIDE_0" :[0.375, -0.171875, 0],    
        "ALPIDE_1" :[0.484375, -0.171875, 0],
        "ALPIDE_2" :[0.46875, -0.1875, 0],
        "ALPIDE_3" :[0.375, -0.15625, 0], 
        "ALPIDE_4" :[0.5, -0.1875, 0]
} #214 events, far too few. less than half of the 80MeV selection

displacements_round6200MeV = {
        "ALPIDE_0" :[0.5, 0.5, 0],    
        "ALPIDE_1" :[0.5, 0.5, 0],
        "ALPIDE_2" :[0.5, -0.5, 0], 
        "ALPIDE_3" :[0.5, 0.5, 0], 
        "ALPIDE_4" :[0.5, -0.5, 0] 
}
 
displacements_round5200MeV = {
        "ALPIDE_0" :[0.5, 0.5, 0],    
        "ALPIDE_1" :[0.0, 0.0, 0],
        "ALPIDE_2" :[0.5, 0.0, 0], 
        "ALPIDE_3" :[0.5, 0.0, 0], 
        "ALPIDE_4" :[0.5, 0.0, 0] 
}

displacements_round4200MeV = {
     'ALPIDE_0': [0.5, -0.25, 0], 
     'ALPIDE_1': [0.0, -0.25, 0], 
     'ALPIDE_2': [0.5, 0.0, 0], 
     'ALPIDE_3': [0.5, 0.0, 0], 
     'ALPIDE_4': [0.5, -0.25, 0]
}

displacements_round3200MeV = {
    'ALPIDE_0': [0.375, -0.25, 0], 
    'ALPIDE_1': [0.375, -0.125, 0], 
    'ALPIDE_2': [0.5, -0.125, 0], 
    'ALPIDE_3': [0.375, 0.0, 0], 
    'ALPIDE_4': [0.5, -0.125, 0]
} 

displacements_round2200MeV = {
    'ALPIDE_0': [0.375, -0.1875, 0], 
    'ALPIDE_1': [0.375, -0.125, 0], 
    'ALPIDE_2': [0.5, -0.1875, 0], 
    'ALPIDE_3': [0.375, 0.0, 0], 
    'ALPIDE_4': [0.5, -0.1875, 0]
} 

displacements_round1200MeV ={
    'ALPIDE_0': [0.375, -0.15625, 0], 
    'ALPIDE_1': [0.46875, -0.15625, 0], 
    'ALPIDE_2': [0.46875, -0.1875, 0], 
    'ALPIDE_3': [0.375, -0.15625, 0], 
    'ALPIDE_4': [0.5, -0.1875, 0]
} 

best_displacement ={
    'ALPIDE_0': [0.35, -0.0625, 0], 
    'ALPIDE_1': [0.5249999999999999, -0.0625, 0], 
    'ALPIDE_2': [0.6, -0.125, 0], 
    'ALPIDE_3': [0.35, 0, 0], 
    'ALPIDE_4': [0.7, -0.0625, 0]
}

best_displacementRefined ={
    'ALPIDE_0': [0.2, 0.034375, 0], 
    'ALPIDE_1': [0.5593750000000001, 0.025, 0], 
    'ALPIDE_2': [0.55, 0.034375, 0], 
    'ALPIDE_3': [0.2, 0.1, 0], 
    'ALPIDE_4': [0.55, 0.034375, 0]
} 

best_displacement_no_multiple_scat ={
    'ALPIDE_0': [0.5, -0.0546875, 0], 
    'ALPIDE_1': [0.65, -0.0546875, 0], 
    'ALPIDE_2': [0.75, -0.06875, 0], 
    'ALPIDE_3': [0.2, -0.125, 0], 
    'ALPIDE_4': [0.5546875, -0.05, 0]
} #Got median DCA  0.05772230 . Got median track RMS  0.48295208  and mean DCA:  0.082  and DCA sigma:  0.130
        

selected_events= []
for event in loadEvents(path_selected):
    for cluster in event.clusters:
        cluster.flipYaxis()
        cluster.alignLocal(best_displacement_no_multiple_scat.get(cluster.detector)) #displacements 6 has no displacement
    selected_events.append(event)
#selected_events = loadEvents(path_selected)

### Add tracks and vertex to events

In [6]:
#path_selected = "/home/cehrich/Software/uits3_krakow22/selectedEvents.pkl" #this was defined already above
events= []
for event in selected_events:
    TrackLeft = Track()
    TrackLeft.fromClusters([event.selectDetector(alpide)[0] for alpide in ["ALPIDE_3","ALPIDE_4"]])
    TrackRight = Track()
    TrackRight.fromClusters([event.selectDetector(alpide)[0] for alpide in ["ALPIDE_0","ALPIDE_1","ALPIDE_2"]])
    vertex = Vertex()
    vertex.fromTracks([TrackLeft,TrackRight])
    event.addTrack(TrackLeft)
    event.addTrack(TrackRight)
    event.vertex = vertex
    events.append(event)

#### Save tracked events to file

In [7]:
path_tracked = "/home/cehrich/Software/uits3_krakow22/trackedEvents.pkl"
save2pickle(events,path_tracked)

## Very clean events

In [91]:
verySelectedEvents = []
firstEvents = 0
for event in events: #loadEvents(path_tracked):
    #200MeV 100e
    if 85 < event.vertex.openingAngle*360/(2*math.pi) < 90 and abs(event.vertex.dca2origin - 18.75) < 2 and event.vertex.dca < 2 and all([5 < cluster.size < 25 for cluster in event.clusters]):#2sigma of 80MeV 100e target sizes
        if all([(-7 < cluster.localPos[0] < 7 and -3 < cluster.localPos[1] < 3) for cluster in event.clusters]):
    #80MeV 100e
    #if 87 < event.vertex.openingAngle*360/(2*math.pi) < 92 and abs(event.vertex.dca2origin - 18.75) < 2 and event.vertex.dca < 2 and all([10 < cluster.size < 30 for cluster in event.clusters]):#2sigma of 80MeV 100e target sizes
            verySelectedEvents.append(event)
            if False: #(firstEvents <= 1):
                print("\nEvent ", firstEvents)#,"\n")
                for cluster in event.clusters:
                    print("Detector: ", cluster.detector, "Global position: ", cluster.globalPos, "Local position: ", cluster.localPos, "size:", cluster.size)#, "\n")
                firstEvents+=1


In [92]:
len(verySelectedEvents)
path_clean = "/home/cehrich/Software/uits3_krakow22/cleanEvents.pkl"
save2pickle(verySelectedEvents,path_clean)

# Plot Vertex

## Try Twisting the Target (rotation about beam axis)

In [10]:
import copy
%jsroot on
hVertexXY = ROOT.TH2F("VertexXY","VertexXY",200,-7,7,200,-7,7)
hVertexXY.GetXaxis().SetTitle("Vertex X position [mm]")
hVertexXY.GetYaxis().SetTitle("Vertex Y position [mm]")
hVertexXZ = ROOT.TH2F("VertexXZ","VertexXZ",120,-2,2,120,-21,-17)
hVertexXZ.GetXaxis().SetTitle("Vertex X position [mm]")
hVertexXZ.GetYaxis().SetTitle("Vertex Z position [mm]")
hVertexYZ = ROOT.TH2F("VertexYZ","VertexYZ",200,-7,7,200,-26,-12)
hVertexYZ.GetXaxis().SetTitle("Vertex Y position [mm]")
hVertexYZ.GetYaxis().SetTitle("Vertex Z position [mm]")
#myVerySelectedEvents=copy.deepcopy(verySelectedEvents)
angleShift=0#-1
for event in selected_events: # myVerySelectedEvents:
    event.vertex.twistVertex(angleShift)
    hVertexXY.Fill(event.vertex.point[0],event.vertex.point[1])
    hVertexXZ.Fill(event.vertex.point[0],event.vertex.point[2])
    hVertexYZ.Fill(event.vertex.point[1],event.vertex.point[2])
canvas = ROOT.TCanvas("","",1200,400)
canvas.Divide(3)
canvas.cd(1)
ROOT.gPad.SetGrid(2)
hVertexXY.Draw("COLZ")
canvas.cd(2)
ROOT.gPad.SetGrid(2)
hVertexXZ.Draw("COLZ")
canvas.cd(3)
ROOT.gPad.SetGrid(2)
hVertexYZ.Draw("COLZ")
canvas.Draw()

Warning in <TROOT::Append>: Replacing existing TH1: VertexXY (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: VertexXZ (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: VertexYZ (Potential memory leak).


In [84]:
def evalDisplacement():
    dcaVals = []
    trackRMSVals = []
    tempEvents=copy.deepcopy(verySelectedEvents) #problem is that the events are changed in the algorithm so a copy must be used
    for event in tempEvents:#loadEvents("/home/cehrich/Software/uits3_krakow22/cleanEvents.pkl"):
        TrackLeft = Track()
        TrackLeft.fromClusters([event.selectDetector(alpide)[0] for alpide in ["ALPIDE_3","ALPIDE_4"]])
        TrackRight = Track()
        TrackRight.fromClusters([event.selectDetector(alpide)[0]for alpide in ["ALPIDE_0","ALPIDE_1","ALPIDE_2"]])
        vertex = Vertex()
        vertex.fromTracks([TrackLeft,TrackRight])
        dcaVals.append(vertex.dca)
        for track in event.tracks:
            if track.nClusters == 3:
                trackRMS = track.rms
                trackRMSVals.append(trackRMS)
    medianDCA = np.median(dcaVals)
    medianTrackRMS = np.median(trackRMSVals)
    
    
    print("Evaluated displacement" ,". Got median DCA ",str(medianDCA)[:10], "mm and mean DCA: ", str(np.mean(dcaVals))[:5], "mm and DCA sigma: ", str(np.std(dcaVals))[:5],"mm. Got median track RMS ",str(medianTrackRMS)[:10], "mm")
    return [medianDCA, medianTrackRMS]

evalDisplacement()

Evaluated displacement . Got median DCA  0.14393487 mm and mean DCA:  0.153 mm and DCA sigma:  0.121 mm. Got median track RMS  0.59359464 mm


[0.14393487101401548, 0.5935946485163659]

#### Vertex positions

#### Position shift between layers

In [10]:
%jsroot on
alpide1 = "ALPIDE_1"
alpide2 = "ALPIDE_2"
hShiftXY = ROOT.TH2F("ShiftXY","ShiftXY",150,-15,15,75,-7.5,7.5)
hShiftX = ROOT.TH1F("ShiftX","ShiftX",150,-15,15)
hShiftY = ROOT.TH1F("ShiftY","ShiftY",75,-7.5,7.5)
for event in myVerySelectedEvents:
    if abs(event.vertex.openingAngle - math.pi/2) < 0.05 and event.vertex.dca < 2:
        hShiftXY.Fill(event.selectDetector(alpide1)[0].localPos[0]-event.selectDetector(alpide2)[0].localPos[0],
                      event.selectDetector(alpide1)[0].localPos[1]-event.selectDetector(alpide2)[0].localPos[1])
        hShiftX.Fill(event.selectDetector(alpide1)[0].localPos[0]-event.selectDetector(alpide2)[0].localPos[0])
        hShiftY.Fill(event.selectDetector(alpide1)[0].localPos[1]-event.selectDetector(alpide2)[0].localPos[1])
canvas = ROOT.TCanvas("","",1200,400)
canvas.Divide(3)
canvas.cd(1)
hShiftXY.Draw("COLZ")
canvas.cd(2)
hShiftX.Draw()
canvas.cd(3)
hShiftY.Draw()
canvas.Draw()

#### Correlation between ALPIDEs after all selections

In [11]:
%jsroot on

nBins = {"x-Axis" : 600, "y-Axis" : 300}
nRange = {"x-Axis" : 15, "y-Axis" : 7.5}
axis = {"x-Axis":0, "y-Axis":1}

detector1 = "ALPIDE_0"
detector2 = "ALPIDE_4"
            
hCorrXX = ROOT.TH2F("Corr_"+detector1+"_x-Axis_"+detector2+"_x-Axis","Corr_"+detector1+"_x-Axis_"+detector2+"_x-Axis", nBins.get("x-Axis"),-nRange.get("x-Axis"),nRange.get("x-Axis"),nBins.get("x-Axis"),-nRange.get("x-Axis"),nRange.get("x-Axis"))
hCorrYY = ROOT.TH2F("Corr_"+detector1+"_y-Axis_"+detector2+"_y-Axis","Corr_"+detector1+"_y-Axis_"+detector2+"_y-Axis", nBins.get("y-Axis"),-nRange.get("y-Axis"),nRange.get("y-Axis"),nBins.get("y-Axis"),-nRange.get("y-Axis"),nRange.get("y-Axis"))
hCorrXX.SetXTitle(detector1 + " - x-Axis [mm]")
hCorrXX.SetYTitle(detector2 + " - x-Axis [mm]")
hCorrYY.SetXTitle(detector1 + " - y-Axis [mm]")
hCorrYY.SetYTitle(detector2 + " - y-Axis [mm]")

for event in events:
    if abs(event.vertex.openingAngle - math.pi/2) < 0.05 and event.vertex.dca < 1:
        for cluster1 in event.selectDetector(detector1):
            for cluster2 in event.selectDetector(detector2):
                hCorrXX.Fill(cluster1.localPos[0],cluster2.localPos[0])
                hCorrYY.Fill(cluster1.localPos[1],cluster2.localPos[1])
                

canvas = ROOT.TCanvas("","",1000,500)
canvas.Divide(2)
canvas.cd(1)
hCorrXX.Draw("COLZ")
canvas.cd(2)
hCorrYY.Draw("COLZ")
canvas.Draw()

# Scan alignment

In [12]:
selectedEvents = [event for event in readEvents("/home/cehrich/Software/uits3_krakow22/selectedEvents.pkl")]

Going to process all events.


In [13]:
detectors = ["ALPIDE_0","ALPIDE_1","ALPIDE_2"]

displacements = [{
    "ALPIDE_0" : [0,0,0], # anchor
    "ALPIDE_1" : [1,0,0],
    "ALPIDE_2" : [0,0,0],
}]

#displacements = [[i/10,j/10] for i in range(10,15,1) for j in range(0,5,1)]
#displacements = [[1.2113099999999999, 0.24715]]
performance = []
for displacement in displacements:
    rmsVals = []
    for event in readEvents("/home/cehrich/Software/uits3_krakow22/selectedEvents.pkl"):
        #event.selectDetector("ALPIDE_1")[0].alignLocal(displacement.get("A"))
        track = Track()
        track.fromClusters([event.selectDetector(alpide)[0].alignLocal(displacement.get(alpide)) for alpide in detectors])
        rmsVals.append(track.rms)
    goodness = np.median(rmsVals)
    performance.append(goodness)
    #print(displacement,":",goodness)

Going to process all events.


Going to process all events.
{'ALPIDE_0': [0, 0, 0], 'ALPIDE_1': [1, 0, 0], 'ALPIDE_2': [0, 0, 0]} : 0.034732187302410344

In [14]:
#plt.hist(rmsVals,1000)

In [15]:
%jsroot on
hRMS = ROOT.TH1F("RMS cluster","RMS Cluster",22*5,-0.1,2.1)
hRMS.GetXaxis().SetTitle("RMS(cluster distance in track) [mm]")
hRMS.GetYaxis().SetTitle("Count")
for event in verySelectedEvents:
    for track in event.tracks:
        if track.nClusters == 3:
            hRMS.Fill(track.rms)
canvas = ROOT.TCanvas()
canvas.SetGridx(2)
hRMS.Draw("")
canvas.Draw()
print("Mean",np.mean(rmsVals),"Med",np.median(rmsVals))

Mean 0.5447511223737146 Med 0.21369702860837494


In [16]:
myCorners = [[i,j] for i in [-1,1] for j in [-1,1]]
myCorners

[[-1, -1], [-1, 1], [1, -1], [1, 1]]

In [17]:
len(loadEvents("/home/cehrich/Software/uits3_krakow22/cleanEvents.pkl"))

3008

In [18]:
def evalDisplacement(displacement):
    dcaVals = []
    tempEvents=copy.deepcopy(verySelectedEvents) #problem is that the events are changed in the algorithm so a copy must be used
    for event in tempEvents:#loadEvents("/home/cehrich/Software/uits3_krakow22/cleanEvents.pkl"):
        TrackLeft = Track()
        TrackLeft.fromClusters([event.selectDetector(alpide)[0].alignLocal(displacement.get(alpide)) for alpide in ["ALPIDE_3","ALPIDE_4"]])
        TrackRight = Track()
        TrackRight.fromClusters([event.selectDetector(alpide)[0].alignLocal(displacement.get(alpide)) for alpide in ["ALPIDE_0","ALPIDE_1","ALPIDE_2"]])
        vertex = Vertex()
        vertex.fromTracks([TrackLeft,TrackRight])
        dcaVals.append(vertex.dca)
    medianDCA = np.median(dcaVals)
    
    
    print("Evaluated ALPIDE_1 at displacement" ,displacement.get("ALPIDE_1") ,". Got median DCA ",str(medianDCA)[:5], " and mean DCA: ", str(np.mean(dcaVals))[:5], " and DCA sigma: ", str(np.std(dcaVals))[:5])
    return medianDCA

def findBest(x1,x2,y1,y2,step):
    print("Round",step)
    corners = [[x,y,0] for x in [x1,x2] for y in [y1,y2]] #problem, this does same thing twice often
    
    
    
    uniqueCorners = set(tuple(row) for row in corners)
    
    
    uniqueCornArr = [list(row) for row in uniqueCorners]
    print(uniqueCornArr)
    
    
    displacements = [{
        "ALPIDE_0" : [0,0,0],# [-0.625, -0.625, 0], #0.19738482268005836
        "ALPIDE_1" : corner, #[0,0,0],#[-0.1875, -0.25, 0], #0.19385599133414377
        "ALPIDE_2" : [0,0,0],#[-0.1875, -0.25, 0], #0.19385599133414377
        "ALPIDE_3" : [0,0,0],#corner,#[0.1875, 0.203125, 0], #with <dca> 0.19476343429968201 
        "ALPIDE_4" : [0,0,0], #[0.0, -0.0078125, 0], #0.19474643045920004
    } for corner in uniqueCornArr]

    results = [evalDisplacement(displacement) for displacement in displacements]
    winner = results.index(min(results)) #lowest index of the lowest median dca Value
    print("Winner of round",step,"is",uniqueCornArr[winner],"with",str(results[winner])[:5])
    if step <= 0 :
        return [results[winner],uniqueCornArr[winner]]
    else : 
        findBest(uniqueCornArr[winner][0],(x1+x2)/2,uniqueCornArr[winner][1],(y1+y2)/2, step-1)
        

    #Formerly converged to:
    
        #"ALPIDE_0" : [0,-1,0],
        #"ALPIDE_1" : [1.03125, -0.96875, 0],
        #"ALPIDE_2" : [0.0, 0.15625, 0],
        #"ALPIDE_3" : corner, #[0.1875, 0.1875,0],
        #"ALPIDE_4" : [0.0, -0.03125, 0],    
    
#findBest(-0.5,0.5,-0.5,0.5,6)

In [19]:
findBest(-1,1,-1,1,6)

Round 6
[[1, -1, 0], [-1, -1, 0], [1, 1, 0], [-1, 1, 0]]
Evaluated ALPIDE_1 at displacement [1, -1, 0] . Got median DCA  0.232  and mean DCA:  0.282  and DCA sigma:  0.258
Evaluated ALPIDE_1 at displacement [-1, -1, 0] . Got median DCA  0.235  and mean DCA:  0.280  and DCA sigma:  0.256
Evaluated ALPIDE_1 at displacement [1, 1, 0] . Got median DCA  0.180  and mean DCA:  0.239  and DCA sigma:  0.269
Evaluated ALPIDE_1 at displacement [-1, 1, 0] . Got median DCA  0.748  and mean DCA:  0.762  and DCA sigma:  0.264
Winner of round 6 is [1, 1, 0] with 0.180
Round 5
[[1, 0.0, 0], [0.0, 0.0, 0], [1, 1, 0], [0.0, 1, 0]]
Evaluated ALPIDE_1 at displacement [1, 0.0, 0] . Got median DCA  0.206  and mean DCA:  0.259  and DCA sigma:  0.261
Evaluated ALPIDE_1 at displacement [0.0, 0.0, 0] . Got median DCA  0.160  and mean DCA:  0.217  and DCA sigma:  0.257
Evaluated ALPIDE_1 at displacement [1, 1, 0] . Got median DCA  0.180  and mean DCA:  0.239  and DCA sigma:  0.269
Evaluated ALPIDE_1 at displaceme

In [20]:
def loadEvents(filename):
    with open(filename, "rb") as f:
        events = pickle.load(f)
        return events

In [21]:
%jsroot on
hVertex = ROOT.TH3F("VertexXYZ","VertexXYZ",100,-6,6, 100,-6,6, 100,-24,-12)
for event in myVerySelectedEvents:
    #if abs(event.vertex.openingAngle - math.pi/2) < 0.015 and event.vertex.dca < 2:
        hVertex.Fill(event.vertex.point[0],event.vertex.point[1],event.vertex.point[2])
    
canvas = ROOT.TCanvas()
hVertex.Draw("BOX3")
canvas.Draw()

#### Opening angle

In [22]:
%jsroot on
hOpeningAngle = ROOT.TH1F("Angle","Angle",300,70,100)
hOpeningAngle.GetXaxis().SetTitle("Opening angle between two tracks #circ")
hOpeningAngle.GetYaxis().SetTitle("Count")
for event in events:
    hOpeningAngle.Fill(event.vertex.openingAngle*360/(2*math.pi))
canvas = ROOT.TCanvas()
canvas.SetGridx(2)
hOpeningAngle.Draw("")
canvas.Draw()

#### DCA between two tracks

In [23]:
%jsroot on
hDCA = ROOT.TH1F("DCA","DCA between tracks",500,-0.1,5.1)
hDCA.GetXaxis().SetTitle("DCA between the tracks from 2 arms [mm]")
hDCA.GetYaxis().SetTitle("Count")
for event in myVerySelectedEvents:
    #if abs(event.vertex.openingAngle - math.pi/2) < 0.05:
    if 86 < event.vertex.openingAngle*360/(2*math.pi) < 90:
        hDCA.Fill(event.vertex.dca)
canvas = ROOT.TCanvas()
canvas.SetGridx(2)
hDCA.Draw()
canvas.Draw()

#### DCA between vertex and [0,0,0]
Should probably use [0,0,-10.2] as origin as 0 along beam axis is currently the middle of the first layer

In [24]:
%jsroot on
hDCA2origin = ROOT.TH1F("DCAtoOrigin","DCA to (0,0,0)",400,12,26)
hDCA2origin.GetXaxis().SetTitle("Distance of vertex to (0,0,0) [mm]")
hDCA2origin.GetYaxis().SetTitle("Count")
for event in events:
    if 86 < event.vertex.openingAngle*360/(2*math.pi) < 90:
        hDCA2origin.Fill(event.vertex.dca2origin)
canvas = ROOT.TCanvas()
canvas.SetGridx(2)
hDCA2origin.Draw()
canvas.Draw()